# Importing python packages

In [ ]:
import glob
from multiprocessing import Pool
import time
from tqdm import tqdm

# Importing SSPARQ setup

In [ ]:
from parameters_py.config import (
					WAVEFORM_DIR,CATALOG_FILE,XML_DIR,SSPARQ_OUTPUT,num_processes
				   )

# Importing SSPARQ functions

In [ ]:
from src.analysis import (
					aic_simple,find_orientation,Braunmiller_Pornsopin_algorithm,calculate_metrics
				   )

from src.utils import (
					quakeml_to_dataframe,moment_tensor_to_nodal_planes,calculate_plunge,mecclass,adjust_baz_for_ZEN,rms,energy
				   )

# Main program

### Reading catalog

In [ ]:
cat = quakeml_to_dataframe(CATALOG_FILE)
cat

### Reading station

In [ ]:
STATIONS_xml = sorted(glob.glob(XML_DIR+'*'))

### Creating input list for one station

In [ ]:
multi_input_list = []
for xml_file in STATIONS_xml:
    input_list = [
    [
        xml_file,
        WAVEFORM_DIR,
        row.depth,
        row.mag,
        row.magType,
        row.time,
        row.latitude,
        row.longitude,
        row.get("moment tensor", []),
        SSPARQ_OUTPUT,
    ]
    for idx,row in cat.iterrows()]
    multi_input_list.append(input_list)


In [ ]:
print("Number of stations:",len(multi_input_list))

### Running multiprocessing function given a input list

In [ ]:
start_time = time.time()

for sta_input in multi_input_list:
    print('Station: ',sta_input[0][0])

    with Pool(processes=num_processes) as p:
        max_ = len(STATIONS_xml)
        with tqdm(total=max_) as pbar:
            for result in p.imap_unordered(calculate_metrics,sta_input):
                pbar.update()

print('\n')
print("--- %.2f execution time (min) ---" % ((time.time() - start_time)/60))
print('\n')